# ReqBrain (Zephyr 7b beta) Download & Setup

In [1]:
# required libraries

# if not installed, use !pip install <name of the API>
# in case of error due to lacking API:
# Python writes the name and command to install the lacking API in the very last lines of the error message.

import torch
import transformers
from torch import cuda, bfloat16

# Resources Checkup

- **Important Note:** A GPU with minimum of 32GB GPU memory is required to load the model!

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()

    # Iterate over each GPU and print its name and memory information
    for i in range(num_gpus):
        gpu = torch.cuda.get_device_properties(i)
        print(f"GPU {i + 1} Name: {gpu.name}")
        print(f"GPU {i + 1} Total Memory: {gpu.total_memory / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. A GPU with minimum of 32GB GPU memory is required to load the model!")

GPU 1 Name: Tesla V100-SXM2-32GB
GPU 1 Total Memory: 31.74 GB


In [3]:
# detecting GPU device to load model on it
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Downloading the Model

In [4]:
model_name = 'kasrahabib/ReqBrain-zephyr-7b-beta'

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast = False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.eval()
model.to(device)
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Model loaded on cuda:0


# Settings

In [5]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [[tokenizer.eos_token], ["<", "|", "user"], ["user", ":"], ["|", "user", "|", ">"]]
]

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [7]:
pipe = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    return_full_text = True, # Set it to True when combining with LangChain
    task='text-generation',
    device = device,
    stopping_criteria = stopping_criteria,  
    temperature = 0.2,
    top_p = 0.15,  
    top_k = 0,  
    max_new_tokens = 512,  
    repetition_penalty = 1.3
)

2024-01-22 13:53:15.675498: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 13:53:15.721817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 13:53:15.721849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 13:53:15.721872: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 13:53:15.741023: I tensorflow/core/platform/cpu_feature_g

# ReqBrain Inferacne 

In [9]:
instruction = "You are a professional requirements engineer who helps users brainstorm more software requirements."

In [10]:
prompt_1 = "I want two usability requirements for bus ticket machines." 

result = result = pipe(f"<|system|>\n{instruction}\n<|user|>\n{prompt_1}\n<|assistant|>\n")
print(result[0]['generated_text'].split("\n<|assistant|>\n")[-1])

/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


1: The machine shall display the message "Please select either a return trip or a one-way trip" if the user tries to purchase a return trip but has not selected a departure stop. 2: The machine shall respond with an error message if the user does not insert enough money into the machine to cover the cost of the tickets.


In [11]:
prompt_2 = "Write me three security requirements for an ATM software?"

result = result = pipe(f"<|system|>\n{instruction}\n<|user|>\n{prompt_2}\n<|assistant|>\n")
print(result[0]['generated_text'].split("\n<|assistant|>\n")[-1])

The Electronic Funds Transfer (EFT) system application shall ensure that only authorized persons can use any given subscriber's account number.
The EFT system application shall ensure that no one can assume another user's identity to access the system.
The EFT system application shall maintain the confidentiality of each subscriber's pin number.


# ReqBrain with LangChain

A basic ReqBrain chat setup setup with LangChain.

In [13]:
# Importing langchain required APIs

from langchain.llms import HuggingFacePipeline
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [14]:
chat_model = HuggingFacePipeline(pipeline = pipe)

memory = ConversationBufferWindowMemory(
    memory_key = "history",
    k = 5,     # Chat history memory, increase/decrease it depending on how far the model should remember!
    return_only_outputs = True)

chat_history = ConversationChain(
    llm = chat_model,
    memory = memory,
    verbose = True)

chat_history.prompt.template = f"{instruction}\n" + """

Current conversation:
{history}
user: {input}
assistant:
"""

In [15]:
def llm_re_elicitor(chat_chain, query):
    chat_chain.predict(input = query)
    last_message = chat_chain.memory.chat_memory.messages[-1]
    last_message_content = last_message.content.split('\n\n')[0].strip()
    
    # cleaning prtompt text
    prompt_id = last_message_content.find('\nuser:')
    if prompt_id != -1:
        last_message_content = last_message_content[:prompt_id]

    # cleaning model generated text
    for stop_token in ['user:', 'assistant:']:
        last_message_content = last_message_content.removesuffix(stop_token)

    return last_message_content.strip()

In [16]:
llm_re_elicitor(chat_history, '''As a software engineer, I want to build an AI chatbot and integrate it with our school's adaptive learning platform. 
The chatbot should be capable of doing content search in the system, doing quizzes, motivating, real-time interaction with students, resolution of misunderstandings, and delivering curriculum content in various formats. 
It also needs the ability to provide topic definitions, connect with a recommendation system, and engage in small talk for a more interactive experience.
I want you to brainstorm requirements for me.''')
print(chat_history.memory.chat_memory.messages[-1].content)



> Entering new ConversationChain chain...
Prompt after formatting:

You are a professional requirements engineer who helps users brainstorm more software requirements.

Current conversation:

### Human: As a software engineer, I want to build an AI chatbot and integrate it with our school's adaptive learning platform. 
The chatbot should be capable of doing content search in the system, doing quizzes, motivating, real-time interaction with students, resolution of misunderstandings, and delivering curriculum content in various formats. 
It also needs the ability to provide topic definitions, connect with a recommendation system, and engage in small talk for a more interactive experience.
I want you to brainstorm requirements for me.
### Assistant: 


/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



> Finished chain.

1 - The chatbot shall perform content search in the Learning Management System (LMS).
2 - The chatbot shall carry out Quizzes.
3 - The chatbot shall motivate students.
4 - The chatbot shall interact with students in real time.
5 - The chatbot shall resolve misunderstandings.
6 - The chatbot shall deliver curriculum content in different formats.
7 - The chatbot shall provide definition topics.
8 - The chatbot shall connect to the Recommendation System.
9 - The chatbot shall have Small Talk capability.
10 - The Chatbot will communicate through multiple communication methods like emails, texts, and social media.
11 - The chatbot must be able to generate responses to student questions automatically.
12 - The chatbot shall convert natural language into ALS (Adaptive Learning Systems) commands.
13 - The chatbot shall determine whether or not to escalate issues to teachers or administrators.
14 - The chatbot shall recommend remedial activities to students.
15 - The chatbot

In [17]:
llm_re_elicitor(chat_history, ''''Concerning requirement number 3, which you proposed in your previous message, tell me two methods I can employ to motivate students?''')
print(chat_history.memory.chat_memory.messages[-1].content)



> Entering new ConversationChain chain...
Prompt after formatting:

You are a professional requirements engineer who helps users brainstorm more software requirements.

Current conversation:
Human: As a software engineer, I want to build an AI chatbot and integrate it with our school's adaptive learning platform. 
The chatbot should be capable of doing content search in the system, doing quizzes, motivating, real-time interaction with students, resolution of misunderstandings, and delivering curriculum content in various formats. 
It also needs the ability to provide topic definitions, connect with a recommendation system, and engage in small talk for a more interactive experience.
I want you to brainstorm requirements for me.
AI: 
1 - The chatbot shall perform content search in the Learning Management System (LMS).
2 - The chatbot shall carry out Quizzes.
3 - The chatbot shall motivate students.
4 - The chatbot shall interact with students in real time.
5 - The chatbot shall resolve

/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/st/st_us-051520/st_ac137798/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



> Finished chain.

Number three require the chatbot to motivation students. Two ways to do this are using gamification techniques or allowing them to learn at their own pace. Number four requires the chatbot to interact with students in real time. Number five requires the chatbot to resolve misunderstandings. Numbers six, ten, eleven, and 12 all require the chat bot to automately answer student questions, generate responses, determine whether or not to escalate issues to teachers or admins, and determine appropriate actions based on student requests.


In [18]:
# put your prtompt between ''' marks for continuation of the chat!

# llm_re_elicitor(chat_history, '''' ''')
# print(chat_history.memory.chat_memory.messages[-1].content)